##### 1. Utility Functions and Imports

In [ ]:
import os
import sys
import random
import shutil
from pathlib import Path
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
import matplotlib.pyplot as plt

#############
# Colab check
#############

def is_colab():
    return 'google.colab' in sys.modules

#############
# Dataset splitting function
#############

def split_dataset(input_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, seed=42):
    random.seed(seed)
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)

    classes = [d.name for d in input_dir.iterdir() if d.is_dir()]

    for cls in classes:
        class_dir = input_dir / cls
        images = list(class_dir.glob('*'))
        random.shuffle(images)

        n_total = len(images)
        n_train = int(train_ratio * n_total)
        n_val = int(val_ratio * n_total)

        splits = {
            'Training': images[:n_train],
            'Validation': images[n_train:n_train + n_val],
            'Testing': images[n_train + n_val:],
        }

        for split_name, split_files in splits.items():
            split_path = output_dir / split_name / cls
            split_path.mkdir(parents=True, exist_ok=True)
            for f in split_files:
                shutil.copy(f, split_path / f.name)

#############
# Model
#############

def compile_and_fit(model,
                    train_data,
                    val_data,
                    model_name,
                    epochs=20,
                    lr=1e-3,
                    checkpoint_dir='./checkpoints'):
    os.makedirs(checkpoint_dir, exist_ok=True)
    ckpt_path = os.path.join(
        checkpoint_dir,
        f"{model_name}" + "_{epoch:02d}-{val_accuracy:.2f}.h5"
    )
    checkpoint_cb = callbacks.ModelCheckpoint(
        filepath=ckpt_path,
        monitor="val_accuracy",
        save_best_only=True,
        verbose=1
    )
    early = callbacks.EarlyStopping(patience=5, restore_best_weights=True)

    model.compile(
        optimizer=optimizers.Adam(learning_rate=lr),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    history = model.fit(
        train_data,
        validation_data=val_data,
        epochs=epochs,
        callbacks=[early, checkpoint_cb]
    )
    return history


#############
# Plot
#############

def plot_history(histories, titles):
    for h, t in zip(histories, titles):
        plt.plot(h.history['val_accuracy'], label=f'{t} val_acc')
    plt.legend()
    plt.show()


#############
# Grad-CAM Heatmaps
#############

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    last_conv_layer = model.get_layer(last_conv_layer_name)
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [last_conv_layer.output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def display_gradcam(img_path, model, last_conv_layer_name):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMAGE_SIZE)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
    orig = tf.keras.preprocessing.image.img_to_array(
        tf.keras.preprocessing.image.load_img(img_path)
    )
    heatmap_resized = tf.image.resize(heatmap[..., tf.newaxis], IMAGE_SIZE).numpy()
    heatmap_col = plt.cm.jet(heatmap_resized.squeeze())[...,:3]
    superimposed = heatmap_col * 0.4 + orig/255.0

    fig, axs = plt.subplots(1,3, figsize=(12,4))
    axs[0].imshow(orig.astype('uint8'))
    axs[0].set_title('Original')
    axs[1].imshow(heatmap, cmap='jet')
    axs[1].set_title('Heatmap')
    axs[2].imshow(superimposed)
    axs[2].set_title('Overlay')
    for ax in axs:
        ax.axis('off')
    plt.show()



##### 2. Setup & Environment check

In [ ]:
if is_colab():
    print("✅ Running in Google Colab")

    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Install and configure Kaggle API
    !pip install -q kaggle
    !mkdir -p ~/.kaggle
    !cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json

    # Download dataset from Kaggle if not already present
    if not os.path.exists("./data/brain_tumor_raw"):
        !kaggle datasets download -d orvile/brain-cancer-mri-dataset
        !unzip -q brain-cancer-mri-dataset.zip -d ./data

    # Set dataset paths
    RAW_DATA_DIR = "/content/drive/MyDrive/datasets/brain_tumor_raw"
    DATA_DIR = "/content/drive/MyDrive/datasets/brain_tumor_split"

    # Only split if not already done
    if not os.path.exists(os.path.join(DATA_DIR, "Training")):
        print("📦 Splitting raw dataset into Training/Validation/Testing...")
        split_dataset(RAW_DATA_DIR, DATA_DIR)
    else:
        print("✅ Pre-split dataset already exists.")
else:
    print("💻 Running locally on Mac")
    DATA_DIR = "/Users/yourname/Projects/brain_tumor_split"  # Replace with your path

##### 3. xxxxxxxxxxx

In [ ]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 4
DATA_DIR = './data/brain_tumor_dataset'

##### 4. Data Loading & Preprocessing

In [ ]:
# Load training + validation from 'Training' directory using an 80/20 split
train_ds = keras.preprocessing.image_dataset_from_directory(
    os.path.join(DATA_DIR, 'Training'),
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = keras.preprocessing.image_dataset_from_directory(
    os.path.join(DATA_DIR, 'Training'),
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

# Load the held-out test set from 'Testing' directory
test_ds = keras.preprocessing.image_dataset_from_directory(
    os.path.join(DATA_DIR, 'Testing'),
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

# Prefetch for performance
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds   = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds  = test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

##### 5. Experiment A: Custom CNNs

In [ ]:
def build_custom_cnn_1(input_shape=(*IMAGE_SIZE,3), num_classes=NUM_CLASSES):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32,3,activation='relu')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64,3,activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128,activation='relu')(x)
    outputs = layers.Dense(num_classes,activation='softmax')(x)
    return models.Model(inputs, outputs)

# A1
cnn1 = build_custom_cnn_1()
history1 = compile_and_fit(cnn1, train_ds, val_ds,
                           model_name="CNN_variant1",
                           epochs=30, lr=1e-3)


def build_custom_cnn_2(input_shape=(*IMAGE_SIZE,3), num_classes=NUM_CLASSES):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64,3,activation='relu')(inputs)
    x = layers.Conv2D(64,3,activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128,3,activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(256,activation='relu')(x)
    outputs = layers.Dense(num_classes,activation='softmax')(x)
    return models.Model(inputs, outputs)

cnn2 = build_custom_cnn_2()
history2 = compile_and_fit(cnn2, train_ds, val_ds,
                           model_name="CNN_variant2",
                           epochs=30, lr=5e-4)


def build_custom_cnn_3(input_shape=(*IMAGE_SIZE,3), num_classes=NUM_CLASSES):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32,3,activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32,3,activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64,3,activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128,activation='relu')(x)
    outputs = layers.Dense(num_classes,activation='softmax')(x)
    return models.Model(inputs, outputs)

cnn3 = build_custom_cnn_3()
history3 = compile_and_fit(cnn3, train_ds, val_ds,
                           model_name="CNN_variant3",
                           epochs=30, lr=1e-3)

##### 6. Experiment B: Transfer Learning

In [ ]:
def build_vgg16(input_shape=(*IMAGE_SIZE,3), num_classes=NUM_CLASSES):
    base = keras.applications.VGG16(include_top=False,
                                    input_shape=input_shape,
                                    weights='imagenet')
    base.trainable = False
    x = layers.GlobalAveragePooling2D()(base.output)
    x = layers.Dense(256,activation='relu')(x)
    outputs = layers.Dense(num_classes,activation='softmax')(x)
    return models.Model(base.input, outputs)

vgg_model = build_vgg16()
history_vgg = compile_and_fit(vgg_model, train_ds, val_ds,
                              model_name="VGG16_finetune",
                              epochs=15, lr=1e-4)


def build_resnet50(input_shape=(*IMAGE_SIZE,3), num_classes=NUM_CLASSES):
    base = keras.applications.ResNet50(include_top=False,
                                       input_shape=input_shape,
                                       weights='imagenet')
    base.trainable = False
    x = layers.GlobalAveragePooling2D()(base.output)
    x = layers.Dense(256,activation='relu')(x)
    outputs = layers.Dense(num_classes,activation='softmax')(x)
    return models.Model(base.input, outputs)

resnet_model = build_resnet50()
history_resnet = compile_and_fit(resnet_model, train_ds, val_ds,
                                 model_name="ResNet50_finetune",
                                 epochs=15, lr=1e-4)

##### 7. Experiment C: Data Augmentation

In [ ]:
data_augment = keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

aug_train_ds = train_ds.map(lambda x, y: (data_augment(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)

# C1
best_cnn_aug = build_custom_cnn_1()
history_aug_custom = compile_and_fit(best_cnn_aug, aug_train_ds, val_ds,
                                     model_name="Aug_CNN_best",
                                     epochs=30, lr=1e-3)

# C2
aug_vgg = build_vgg16()
history_aug_vgg = compile_and_fit(aug_vgg, aug_train_ds, val_ds,
                                  model_name="Aug_VGG16",
                                  epochs=15, lr=1e-4)

# C3
aug_resnet = build_resnet50()
history_aug_resnet = compile_and_fit(aug_resnet, aug_train_ds, val_ds,
                                     model_name="Aug_ResNet50",
                                     epochs=15, lr=1e-4)

In [ ]:
# Evaluate best models on test set
for name, model in [
    ("Best_Custom_CNN", best_custom_cnn),
    ("VGG16", vgg_model),
    ("ResNet50", resnet_model)
]:
    loss, acc = model.evaluate(test_ds)
    print(f"{name} test accuracy: {acc:.3f}")

##### 8. Evaluation & Visualization

In [ ]:
plot_history(
    [history1, history2, history3,
     history_vgg, history_resnet,
     history_aug_custom, history_aug_vgg, history_aug_resnet],
    ['CNN1','CNN2','CNN3','VGG','ResNet',
     'Aug-CNN','Aug-VGG','Aug-ResNet'])